## 1) UFC Events Snapshot — URLs & Details  
**Purpose:** Crawl the UFCStats "Completed Events" archive, capture event URLs and metadata (name, date, location, org IDs).  
**Inputs / Config:** Static listing page (`ufcstats.com/statistics/events/completed?page=all`) with retry/session settings.  
**Outputs:** A dataframe of events with normalized fields (incl. resolved URLs) to be used by downstream scrapers.  
**Notes:** Idempotent; uses polite retry/backoff. Handles minor HTML structure drift.

In [1]:
# === UFC Events Snapshot Scraper (URLs + Details) ===

from datetime import datetime
from dataclasses import dataclass
from typing import List, Optional, Tuple
import pandas as pd
import requests, logging, time, os, sys
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# --- Config ---
UFCSTATS_URL = "http://ufcstats.com/statistics/events/completed?page=all"
TIMEOUT = 15
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; UFC-AI/1.0; +https://example.com/bot)"}
OUTDIR = "/kaggle/working"  # Kaggle default output dir
os.makedirs(OUTDIR, exist_ok=True)

# Optional: limit liczby eventów (None = wszystkie)
EVENT_LIMIT = None  # np. 200 dla szybkiego testu

# --- Logging ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    stream=sys.stdout
)

def http_session() -> requests.Session:
    s = requests.Session()
    retry = Retry(
        total=4,                 # 1 żądanie + 3 retrysy
        backoff_factor=0.8,      # 0.8s, 1.6s, 3.2s...
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"])
    )
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

def save_df(df: pd.DataFrame, name: str):
    """Zapisuje do CSV i Parquet w /kaggle/working"""
    csv_path = os.path.join(OUTDIR, f"{name}.csv")
    pq_path  = os.path.join(OUTDIR, f"{name}.parquet")
    # CSV
    df.to_csv(csv_path, index=False)
    # Parquet
    try:
        df.to_parquet(pq_path, index=False)  # pyarrow jest na Kaggle
    except Exception as e:
        logging.warning(f"Parquet save failed ({e}); trying fastparquet…")
        df.to_parquet(pq_path, index=False, engine="fastparquet")
    logging.info(f"Saved: {csv_path}  &  {pq_path}")

def parse_events(html: str) -> pd.DataFrame:
    """Parsuje listę eventów (URL, Date, Title) ze strony zbiorczej."""
    soup = BeautifulSoup(html, "html.parser")
    rows = [
        row for row in soup.select("tr.b-statistics__table-row")
        if row.select_one("a") and row.select_one("span")
    ]

    events: List[Tuple[str, datetime, str]] = []
    skipped = 0
    for row in rows:
        a_tag = row.select_one("a")
        date_span = row.select_one("span")

        event_url = (a_tag["href"] or "").strip().rstrip("/") if a_tag else ""
        event_title = (a_tag.text or "").strip() if a_tag else ""
        date_txt = (date_span.text or "").strip() if date_span else ""

        try:
            event_date = datetime.strptime(date_txt, "%B %d, %Y").date()
        except Exception as e:
            logging.warning(f"[parse] skip row — bad date '{date_txt}': {e}")
            skipped += 1
            continue

        if not event_url or not event_title:
            skipped += 1
            continue

        events.append((event_url, event_date, event_title))

    logging.info(f"[parse] OK: {len(events)}, skipped: {skipped}")

    if not events:
        return pd.DataFrame(columns=["event_url", "event_date", "event_title"])

    df = pd.DataFrame(events, columns=["event_url", "event_date", "event_title"])
    df["event_url"] = df["event_url"].astype(str).str.strip().str.rstrip("/")
    df["event_title"] = df["event_title"].astype(str).str.strip()
    df["event_date"] = pd.to_datetime(df["event_date"]).dt.date
    return df

def parse_event_details(html: str) -> Tuple[Optional[str], Optional[datetime], Optional[str], Optional[str], Optional[str]]:
    """Zwraca (event_name, event_date, city, state, country)"""
    soup = BeautifulSoup(html, "html.parser")

    # Nazwa eventu
    name_tag = soup.find("h2", class_="b-content__title")
    event_name = name_tag.get_text(strip=True) if name_tag else None

    details_box = soup.find("div", class_="b-list__info-box")
    event_date = None
    location_city = location_state = location_country = None

    if details_box:
        for li in details_box.find_all("li"):
            text = li.get_text(strip=True)
            if "Date:" in text:
                date_text = text.split("Date:", 1)[1].strip()
                try:
                    event_date = datetime.strptime(date_text, "%B %d, %Y").date()
                except Exception:
                    event_date = None
            if "Location:" in text:
                location_text = text.split("Location:", 1)[1].strip()
                parts = [p.strip() for p in location_text.split(",")]
                location_city = parts[0] if len(parts) > 0 else None
                location_state = parts[1] if len(parts) > 1 else None
                location_country = parts[2] if len(parts) > 2 else None

    return event_name, event_date, location_city, location_state, location_country


## 2) Orchestrate: Run Events Snapshot  
**Purpose:** Execute the event snapshot end‑to‑end: request listing page, parse rows, resolve individual event pages if needed.  
**Inputs:** Events listing URL, HTTP headers, timeouts, retry policy.  
**Outputs:** In‑memory dataframe of events; optionally persisted for later cells.  
**Notes:** Basic validation of HTTP status and non‑empty results.

In [2]:
# Cell 2: Run full snapshot (events URLs + event details) and save as CSV & Parquet

from tqdm import tqdm

session = http_session()

# 1) Pobierz stronę zbiorczą eventów
logging.info("Fetching events index…")
resp = session.get(UFCSTATS_URL, headers=HEADERS, timeout=TIMEOUT)
if resp.status_code != 200:
    raise RuntimeError(f"UFCStats HTTP {resp.status_code}")

df_events_urls = parse_events(resp.text)

if df_events_urls.empty:
    logging.warning("Brak zparsowanych eventów — możliwa zmiana layoutu ufcstats.com")
else:
    if EVENT_LIMIT is not None:
        df_events_urls = df_events_urls.head(int(EVENT_LIMIT))
        logging.info(f"EVENT_LIMIT active → using first {len(df_events_urls)} events")
    save_df(df_events_urls, "UFC_events_urls")

# 2) Dla każdego event_url pobierz szczegóły eventu
rows = []
logging.info(f"Fetching details for {len(df_events_urls)} events…")
for url, date_, title in tqdm(df_events_urls[["event_url", "event_date", "event_title"]].itertuples(index=False), total=len(df_events_urls)):
    try:
        r = session.get(url, headers=HEADERS, timeout=TIMEOUT)
        r.raise_for_status()
        name, ev_date, city, state, country = parse_event_details(r.text)

        # Walidacja minimum
        if not name:
            logging.warning(f"[details] skip {url} — missing name")
            continue
        final_date = ev_date if ev_date is not None else date_

        rows.append({
            "event_url": url,
            "event_name": name,
            "event_date": final_date,     # DATE
            "event_city": city,
            "event_state": state,
            "event_country": country,
            "event_title_index": title    # tytuł z listy (czasem różni się od h2)
        })

        # miękki throttling (uprzejmość dla serwisu)
        time.sleep(0.05)

    except Exception as e:
        logging.warning(f"[details] error {url}: {e}")
        continue

df_events_data = pd.DataFrame(rows, columns=[
    "event_url","event_name","event_date","event_city","event_state","event_country","event_title_index"
])

if df_events_data.empty:
    logging.warning("No valid event details parsed.")
else:
    # typy
    df_events_data["event_url"] = df_events_data["event_url"].astype(str).str.strip().str.rstrip("/")
    df_events_data["event_name"] = df_events_data["event_name"].astype(str).str.strip()
    df_events_data["event_date"] = pd.to_datetime(df_events_data["event_date"]).dt.date

    save_df(df_events_data, "UFC_events_data")

# Podsumowanie
print("Summary:")
print(" - UFC_events_urls:", df_events_urls.shape)
print(" - UFC_events_data:", df_events_data.shape)


100%|██████████| 744/744 [05:16<00:00,  2.35it/s]

Summary:
 - UFC_events_urls: (744, 3)
 - UFC_events_data: (744, 7)


## 3) UFC Fight URLs Snapshot  
**Purpose:** For each archived event, collect links to all fight pages (one URL per bout).  
**Inputs:** Event URLs from the previous snapshot.  
**Outputs:** A deduplicated dataframe of fight URLs with event context (event_id, card order, etc.).  
**Notes:** Resilient to missing or cancelled bouts; graceful handling of pagination quirks.

In [3]:
# === UFC Fight URLs Snapshot ===
# Wejście: /kaggle/working/UFC_events_urls.(csv|parquet)
# Wyjście: /kaggle/working/UFC_fights_urls.(csv|parquet)

import os, sys, logging, time
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tqdm import tqdm

# --- I/O ---
OUTDIR = "/kaggle/working"
EVENTS_CSV = os.path.join(OUTDIR, "UFC_events_urls.csv")
EVENTS_PARQUET = os.path.join(OUTDIR, "UFC_events_urls.parquet")
FIGHTS_URLS_BASENAME = "UFC_fights_urls"

# --- Logging ---
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s", stream=sys.stdout)

# --- Helpers ---
def save_df(df: pd.DataFrame, name: str):
    csv_path = os.path.join(OUTDIR, f"{name}.csv")
    pq_path  = os.path.join(OUTDIR, f"{name}.parquet")
    df.to_csv(csv_path, index=False)
    try:
        df.to_parquet(pq_path, index=False)
    except Exception as e:
        logging.warning(f"Parquet save failed ({e}); trying fastparquet…")
        df.to_parquet(pq_path, index=False, engine="fastparquet")
    logging.info(f"Saved: {csv_path}  &  {pq_path}")

def http_session():
    s = requests.Session()
    retry = Retry(
        total=4,
        backoff_factor=0.8,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"])
    )
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

def normalize_url(u: str) -> str:
    return (u or "").strip().rstrip("/")

# --- Load events (prefer Parquet if present) ---
if os.path.exists(EVENTS_PARQUET):
    events_df = pd.read_parquet(EVENTS_PARQUET)
elif os.path.exists(EVENTS_CSV):
    events_df = pd.read_csv(EVENTS_CSV)
else:
    raise FileNotFoundError("Nie znaleziono pliku UFC_events_urls.(csv|parquet) w /kaggle/working")

# sanity & normalization
events_df = events_df.rename(columns={"Event_url": "event_url", "Date": "event_date", "Title": "event_title"})
events_df["event_url"] = events_df["event_url"].astype(str).str.strip().str.rstrip("/")
events_df["event_title"] = events_df["event_title"].astype(str).str.strip()
events_df["event_date"] = pd.to_datetime(events_df["event_date"]).dt.date

# Optional: ogranicz liczbę eventów przy testach
EVENT_LIMIT = None  # np. 50
if EVENT_LIMIT is not None:
    events_df = events_df.head(int(EVENT_LIMIT))
    logging.info(f"EVENT_LIMIT active → using first {len(events_df)} events")

# --- Scrape fights per event ---
session = http_session()
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; UFC-AI/1.0; +https://example.com/bot)"}

records = []
logging.info(f"Scraping fights for {len(events_df)} events…")
for row in tqdm(events_df.itertuples(index=False), total=len(events_df)):
    event_url = normalize_url(row.event_url)
    try:
        r = session.get(event_url, headers=HEADERS, timeout=15)
        if r.status_code != 200:
            logging.warning(f"[{r.status_code}] {event_url}")
            continue
        soup = BeautifulSoup(r.text, "html.parser")
        rows = soup.select("tr.b-fight-details__table-row")
        for tr in rows:
            cols = tr.find_all("td")
            if len(cols) < 2:
                continue
            a = cols[0].find("a", href=True)
            if not a:
                continue
            fight_url = normalize_url(a["href"])
            records.append({
                "Event_URL": event_url,
                "Title": row.event_title,
                "Date": pd.to_datetime(row.event_date).strftime("%Y-%m-%d"),
                "Fight_URL": fight_url
            })
        time.sleep(0.03)  # uprzejmy throttling
    except Exception as e:
        logging.warning(f"[fight_urls] {event_url} → {e}")
        continue

# --- Build DF, dedupe & save ---
if not records:
    logging.warning("Brak zebranych walk (records = []).")
    fights_urls_df = pd.DataFrame(columns=["Event_URL","Title","Date","Fight_URL"])
else:
    fights_urls_df = pd.DataFrame(records)
    fights_urls_df["Event_URL"] = fights_urls_df["Event_URL"].astype(str).str.strip().str.rstrip("/")
    fights_urls_df["Fight_URL"] = fights_urls_df["Fight_URL"].astype(str).str.strip().str.rstrip("/")
    fights_urls_df = fights_urls_df.drop_duplicates(subset=["Fight_URL"]).reset_index(drop=True)

save_df(fights_urls_df, FIGHTS_URLS_BASENAME)

print("Summary:")
print(" - UFC_fights_urls:", fights_urls_df.shape)


100%|██████████| 744/744 [05:03<00:00,  2.45it/s]

Summary:
 - UFC_fights_urls: (8324, 4)


## 4) UFC Fights DATA Snapshot  
**Purpose:** Visit each fight page and extract structured fight metadata (fighters, weight class, method, rounds, refs, time).  
**Inputs:** Fight URLs dataframe.  
**Outputs:** Fight‑level dataset ready for joining with per‑fighter stats.  
**Notes:** Normalizes “end method” and time formats; defensive parsing for corner cases (NC, DQ, overturned).

In [4]:
# === UFC Fights DATA Snapshot ===
# Wejście: /kaggle/working/UFC_fights_urls.(csv|parquet)
# Wyjście: /kaggle/working/UFC_fights_data.(csv|parquet)

import os, sys, logging, re, time
import pandas as pd
import requests, bs4
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

OUTDIR = "/kaggle/working"
FIGHTS_URLS_CSV = os.path.join(OUTDIR, "UFC_fights_urls.csv")
FIGHTS_URLS_PARQUET = os.path.join(OUTDIR, "UFC_fights_urls.parquet")
FIGHTS_DATA_BASENAME = "UFC_fights_data"

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s", stream=sys.stdout)

def save_df(df: pd.DataFrame, name: str):
    csv_path = os.path.join(OUTDIR, f"{name}.csv")
    pq_path  = os.path.join(OUTDIR, f"{name}.parquet")
    df.to_csv(csv_path, index=False)
    try:
        df.to_parquet(pq_path, index=False)
    except Exception as e:
        logging.warning(f"Parquet save failed ({e}); trying fastparquet…")
        df.to_parquet(pq_path, index=False, engine="fastparquet")
    logging.info(f"Saved: {csv_path}  &  {pq_path}")

def http_session():
    s = requests.Session()
    retry = Retry(
        total=4, backoff_factor=0.7,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"])
    )
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

def normalize_url(u: str) -> str:
    return (u or "").strip().rstrip("/")

def get_info(label, soup):
    for p in soup.select("p.b-fight-details__text"):
        for tag in p.find_all("i", class_="b-fight-details__label"):
            if tag.text.strip() == label:
                nxt = tag.next_sibling
                while nxt:
                    if isinstance(nxt, str):
                        t = nxt.strip()
                        if t: return t
                    elif hasattr(nxt, "get_text"):
                        t = nxt.get_text(strip=True)
                        if t: return t
                    nxt = nxt.next_sibling
    return None

def get_result_details(soup):
    for p in soup.select("p.b-fight-details__text"):
        if "Details:" in p.text:
            text = p.get_text(separator=" ", strip=True)
            return text.split("Details:")[-1].strip()
    return None

def get_event_name(soup):
    header = soup.select_one("h2.b-content__title")
    return header.get_text(strip=True).replace("Event:", "").strip() if header else None

def get_winner(f1, f2, fighters):
    s1 = fighters[0].select_one(".b-fight-details__person-status")
    s2 = fighters[1].select_one(".b-fight-details__person-status")
    t1 = s1.get_text(strip=True).upper() if s1 else ""
    t2 = s2.get_text(strip=True).upper() if s2 else ""
    if t1 == "W": return f1
    if t2 == "W": return f2
    return "No Contest"

# --- Load fights URLs ---
if os.path.exists(FIGHTS_URLS_PARQUET):
    fights_urls_df = pd.read_parquet(FIGHTS_URLS_PARQUET)
elif os.path.exists(FIGHTS_URLS_CSV):
    fights_urls_df = pd.read_csv(FIGHTS_URLS_CSV)
else:
    raise FileNotFoundError("Nie znaleziono pliku UFC_fights_urls.(csv|parquet) w /kaggle/working")

fights_urls_df = fights_urls_df.rename(columns={"Fight_URL":"fight_url"})
fights_urls_df["fight_url"] = fights_urls_df["fight_url"].astype(str).str.strip().str.rstrip("/")
fight_urls = fights_urls_df["fight_url"].dropna().unique().tolist()

# Optional: ogranicz liczbę przy testach
FIGHT_LIMIT = None  # np. 1000
if FIGHT_LIMIT is not None:
    fight_urls = fight_urls[:int(FIGHT_LIMIT)]
    logging.info(f"FIGHT_LIMIT active → using first {len(fight_urls)} fights")

session = http_session()
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; UFC-AI/1.0)"}

def scrape_one(url: str):
    url = normalize_url(url)
    try:
        resp = session.get(url, headers=HEADERS, timeout=15)
        resp.raise_for_status()
        soup = bs4.BeautifulSoup(resp.text, "html.parser")

        fighters = soup.select("div.b-fight-details__person")
        if len(fighters) != 2:
            return None

        f1 = fighters[0].select_one("h3")
        f2 = fighters[1].select_one("h3")
        f1_name = f1.get_text(strip=True) if f1 else None
        f2_name = f2.get_text(strip=True) if f2 else None

        a1 = fighters[0].select_one("a")
        a2 = fighters[1].select_one("a")
        f1_url = normalize_url(a1["href"]) if a1 and a1.has_attr("href") else None
        f2_url = normalize_url(a2["href"]) if a2 and a2.has_attr("href") else None

        if not f1_name or not f2_name:
            return None

        winner = get_winner(f1_name, f2_name, fighters)
        referee = get_info("Referee:", soup)

        # num_rounds z "Time format: 5 Rnd (5-5-5-5-5)"
        num_rounds = 0
        tf = next((p for p in soup.select("p.b-fight-details__text") if "Time format:" in p.text), None)
        if tf:
            m = re.search(r'(\d+)\s*Rnd', tf.get_text(" ", strip=True))
            if m:
                try:
                    num_rounds = int(m.group(1))
                except ValueError:
                    num_rounds = 0

        # meta
        try:
            finish_round = int(get_info("Round:", soup) or 0)
        except ValueError:
            finish_round = 0
        finish_time = get_info("Time:", soup)

        wt = soup.select_one("i.b-fight-details__fight-title")
        weight_class = wt.get_text(strip=True).replace(" Bout", "") if wt else None
        title_fight = bool(weight_class and "Title Bout" in weight_class)
        gender = "F" if (weight_class and "Women" in weight_class) else "M"

        result = get_info("Method:", soup)
        result_details = get_result_details(soup)
        event_name = get_event_name(soup)

        return {
            "event_name": event_name,
            "referee": referee,
            "f_1": f1_name,
            "f_2": f2_name,
            "f_1_url": f1_url,
            "f_2_url": f2_url,
            "winner": winner,
            "num_rounds": num_rounds,
            "title_fight": title_fight,
            "weight_class": weight_class,
            "gender": gender,
            "result": result,
            "result_details": result_details,
            "finish_round": finish_round,
            "finish_time": finish_time,
            "fight_url": url
        }
    except Exception:
        return None

# --- Parallel scrape (bez przesady, żeby być "uprzejmym") ---
MAX_WORKERS = 16  # możesz zwiększyć do 24/32, ale 16 jest zwykle bezpieczne
results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(scrape_one, u) for u in fight_urls]
    for f in tqdm(as_completed(futures), total=len(futures)):
        rec = f.result()
        if rec:
            results.append(rec)

# --- Build DF & save ---
cols = ["event_name","referee","f_1","f_2","f_1_url","f_2_url","winner","num_rounds",
        "title_fight","weight_class","gender","result","result_details",
        "finish_round","finish_time","fight_url"]
if not results:
    logging.warning("Brak rekordów (results = []).")
    fights_df = pd.DataFrame(columns=cols)
else:
    fights_df = pd.DataFrame(results)[cols].copy()
    fights_df["fight_url"] = fights_df["fight_url"].astype(str).str.strip().str.rstrip("/")
    fights_df["f_1_url"] = fights_df["f_1_url"].astype(str)
    fights_df["f_2_url"] = fights_df["f_2_url"].astype(str)
    fights_df = fights_df.drop_duplicates(subset=["fight_url"]).reset_index(drop=True)

save_df(fights_df, FIGHTS_DATA_BASENAME)

print("Summary:")
print(" - UFC_fights_data:", fights_df.shape)


100%|██████████| 8324/8324 [12:17<00:00, 11.29it/s]


Summary:
 - UFC_fights_data: (8241, 16)


## 5) Scrape UFC Fight **Stats** (async, aiohttp)  
**Purpose:** Collect per‑fighter, per‑round statistics from the UFCStats tables for each fight URL, concurrently.  
**Inputs:** Fight URLs; async concurrency/timeout settings.  
**Outputs:** Long‑format stats table (fighter x round) covering totals and significant strike breakdowns.  
**Notes:** Uses aiohttp with semaphore‑controlled concurrency and retries. Robust to partial tables and missing rounds.

In [6]:
# %% [markdown]
# ## Scrape: UFC fight *stats* (async, aiohttp) → CSV + Parquet

# %%
import asyncio
import aiohttp
from aiohttp import ClientTimeout
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import time
import traceback
import re
from tqdm.auto import tqdm
from pathlib import Path

# ---------- Config ----------
INPUT_FIGHTS_URLS_PARQUET = "UFC_fights_urls.parquet"
INPUT_FIGHTS_URLS_CSV     = "UFC_fights_urls.csv"
OUTPUT_STATS_CSV          = "UFC_fights_stats_data.csv"
OUTPUT_STATS_PARQUET      = "UFC_fights_stats_data.parquet"

CONCURRENCY = 8          # równoległość requestów (bezpiecznie dla ufcstats)
RETRIES     = 3          # próby / URL
DELAY_RANGE = (0.35, 1.0) # jitter (throttling)
REQUEST_TIMEOUT = 18

# ---------- I/O: wczytaj listę walk ----------
def load_fight_urls() -> pd.DataFrame:
    if Path(INPUT_FIGHTS_URLS_PARQUET).exists():
        df = pd.read_parquet(INPUT_FIGHTS_URLS_PARQUET)
    elif Path(INPUT_FIGHTS_URLS_CSV).exists():
        df = pd.read_csv(INPUT_FIGHTS_URLS_CSV)
    else:
        raise FileNotFoundError("Nie znaleziono pliku z linkami walk: "
                                f"{INPUT_FIGHTS_URLS_PARQUET} ani {INPUT_FIGHTS_URLS_CSV}")

    col = None
    for c in df.columns:
        if c.lower() in {"fight_url","fight url","url","link"}:
            col = c
            break
    if col is None and "Fight_URL" in df.columns:
        col = "Fight_URL"
    if col is None:
        # spróbuj po nazwie z poprzednich kroków
        candidates = [c for c in df.columns if "fight" in c.lower() and "url" in c.lower()]
        if not candidates:
            raise ValueError("Nie znalazłem kolumny z URL-ami walk.")
        col = candidates[0]

    df[col] = df[col].astype(str).str.strip().str.rstrip("/")
    df = df.dropna(subset=[col]).drop_duplicates(subset=[col]).reset_index(drop=True)
    df = df.rename(columns={col: "fight_url"})
    return df[["fight_url"]]

# ---------- Parser utils ----------
def time_to_sec(val: str) -> int:
    try:
        m, s = map(int, val.strip().split(":"))
        return m * 60 + s
    except:
        return 0

def split_of(txt: str):
    try:
        a, b = txt.split(" of ")
        return int(a.strip()), int(b.strip())
    except:
        return 0, 0

def to_int(txt: str) -> int:
    try:
        return int(txt.strip().replace("%","").replace("---","0"))
    except:
        return 0

def find_summary_table(soup: BeautifulSoup):
    p_tag = soup.find("p", class_="b-fight-details__collapse-link_tot", string=re.compile("Totals", re.IGNORECASE))
    if not p_tag:
        return None
    return p_tag.find_next("table")

def parse_round_table(soup: BeautifulSoup, idx: int, f1: str, f2: str,
                      url: str, u1: str, u2: str, cols: list) -> pd.DataFrame:
    tables = soup.select("table.b-fight-details__table.js-fight-table")
    if idx >= len(tables):
        return pd.DataFrame()
    rows = tables[idx].find_all(["thead", "tr"])
    round_n = 0
    out = []

    for row in rows:
        if row.name == "thead" and "Round" in row.text:
            match = re.search(r"Round (\d+)", row.text)
            if match:
                round_n = int(match.group(1))
            continue
        if row.name != "tr":
            continue

        tds = row.find_all("td")
        if not tds or len(tds) < len(cols):
            continue

        p_tags_by_td = [td.find_all("p") for td in tds]
        for i, (f_name, f_url) in enumerate([(f1, u1), (f2, u2)]):
            r = {
                "fighter_id": f_name,
                "fighter_url": f_url,
                "fight_url": url,
                "round": round_n
            }
            for j, key in enumerate(cols):
                try:
                    txt = p_tags_by_td[j+1][i].text.strip()
                    if "of" in txt:
                        a, b = txt.split(" of ")
                        r[f"{key}_succ"] = int(a.strip())
                        r[f"{key}_att"]  = int(b.strip())
                    elif ":" in txt:
                        r[key] = time_to_sec(txt)
                    else:
                        r[key] = to_int(txt)
                except:
                    r[key] = 0
            out.append(r)

    return pd.DataFrame(out)

def get_texts_from_summary_row(summary_tds):
    # summary_tds to lista <td> w pierwszym wierszu sekcji Totals
    def get_p(col_idx, i):
        try:
            return summary_tds[col_idx].find_all("p")[i].text.strip()
        except:
            return "0"
    return get_p

async def fetch_fight(session, url, sem,
                      delay_range=DELAY_RANGE,
                      max_retries=RETRIES) -> list[dict]:
    async with sem:
        for attempt in range(max_retries):
            try:
                await asyncio.sleep(random.uniform(*delay_range))
                async with session.get(url, timeout=ClientTimeout(total=REQUEST_TIMEOUT)) as res:
                    html = await res.text()
                    soup = BeautifulSoup(html, "html.parser")

                    # identyfikacja zawodników
                    names = soup.select("a.b-fight-details__person-link")
                    if len(names) < 2:
                        return []

                    f1, f2 = names[0].text.strip(), names[1].text.strip()
                    u1, u2 = names[0].get("href","").strip().rstrip("/"), names[1].get("href","").strip().rstrip("/")

                    # tabela Totals
                    summary_table = find_summary_table(soup)
                    if not summary_table:
                        return []

                    t_body_rows = summary_table.select("tbody tr")
                    if not t_body_rows:
                        return []

                    cols = t_body_rows[0].find_all("td")
                    get_p = get_texts_from_summary_row(cols)

                    # rekordy „Totals” per fighter
                    records = []
                    for i, (f_name, f_url) in enumerate([(f1,u1),(f2,u2)]):
                        sig_succ, sig_att = split_of(get_p(2, i))
                        tot_succ, tot_att = split_of(get_p(4, i))
                        td_succ,  td_att  = split_of(get_p(5, i))

                        rec = {
                            "fighter_id": f_name,
                            "fighter_url": f_url,
                            "fight_url": url,
                            "knockdowns": to_int(get_p(1, i)),
                            "sig_strikes_succ": sig_succ,
                            "sig_strikes_att": sig_att,
                            "total_strikes_succ": tot_succ,
                            "total_strikes_att": tot_att,
                            "takedown_succ": td_succ,
                            "takedown_att": td_att,
                            "submission_att": to_int(get_p(7, i)),
                            "reversals": to_int(get_p(8, i)),
                            "ctrl_time_sec": time_to_sec(get_p(9, i)),
                        }
                        records.append(rec)

                    # tabele rundowe
                    df_totals = parse_round_table(
                        soup, 0, f1, f2, url, u1, u2,
                        ["knockdowns","sig_strikes","sig_pct","total_strikes","td_1","td_2","submission_att","reversals","ctrl"]
                    )
                    df_sig = parse_round_table(
                        soup, 1, f1, f2, url, u1, u2,
                        ["sig_pct","sig_strikes","head","body","leg","distance","clinch","ground"]
                    )

                    # index po (fighter_url, round) dla łatwego łączenia
                    if not df_totals.empty:
                        df_totals = df_totals.set_index(["fighter_url","round"])
                    if not df_sig.empty:
                        df_sig = df_sig.set_index(["fighter_url","round"])

                    # wzbogacenie rekordów totals o rundy 1..5
                    enriched = []
                    all_keys = set()
                    for rec in records:
                        f_url = rec["fighter_url"]
                        base = rec.copy()
                        for r in range(1, 6):
                            row_t = df_totals.loc[(f_url, r)].to_dict() if (not df_totals.empty and (f_url, r) in df_totals.index) else {}
                            row_s = df_sig.loc[(f_url, r)].to_dict() if (not df_sig.empty and (f_url, r) in df_sig.index) else {}

                            for k, v in row_t.items():
                                col = f"round{r}_{k}"
                                base[col] = v
                                all_keys.add(col)
                            for k, v in row_s.items():
                                col = f"round{r}_{k}"
                                base[col] = v
                                all_keys.add(col)

                        # dopełnij brakujące kolumny zerami
                        for col in all_keys:
                            base.setdefault(col, 0)

                        enriched.append(base)

                    return enriched

            except Exception:
                # log lokalny; w notebooku nie rozwlekamy outputu
                await asyncio.sleep(1.5 * (attempt + 1))

        # po RETRIES się poddajemy
        return []

async def scrape_stats_async(urls: list[str]) -> list[dict]:
    sem = asyncio.Semaphore(CONCURRENCY)
    headers = {"User-Agent": "Mozilla/5.0 (compatible; Kaggle-UFC-Scraper/1.0)"}
    timeout = ClientTimeout(total=REQUEST_TIMEOUT)

    results = []
    connector = aiohttp.TCPConnector(limit=CONCURRENCY*3, enable_cleanup_closed=True)

    async with aiohttp.ClientSession(headers=headers, timeout=timeout, connector=connector) as session:
        tasks = [fetch_fight(session, u, sem) for u in urls]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Scraping fights (stats)"):
            res = await f
            if res:
                results.extend(res)
    return results


## 6) Orchestrate: Stats Job Runner  
**Purpose:** Load fight URLs, spawn the async stats scraper, and combine outputs into a single dataframe.  
**Inputs:** Source paths produced by earlier cells; concurrency controls.  
**Outputs:** Final fight‑stats dataframe; optional CSV/Parquet persistence.  
**Notes:** Includes fallback to `nest_asyncio` for Kaggle/Notebook event‑loop edge cases.

In [8]:
# %%
# 1) urls
df_urls = load_fight_urls()
fight_urls = df_urls["fight_url"].tolist()
print(f"Fight URLs total: {len(fight_urls)}")
# --- notebook-safe async runner ---
def run_async_notebook_safe(coro):
    try:
        return asyncio.run(coro)
    except RuntimeError as e:
        # Jupyter/Kaggle ma już działający event loop
        if "asyncio.run() cannot be called from a running event loop" in str(e):
            try:
                import nest_asyncio  # zwykle dostępne w Kaggle
                nest_asyncio.apply()
            except Exception:
                pass
            loop = asyncio.get_event_loop()
            return loop.run_until_complete(coro)
        raise
# 2) run
start = time.time()
records = run_async_notebook_safe(scrape_stats_async(fight_urls))
elapsed = time.time() - start

# 3) do DataFrame
if not records:
    raise RuntimeError("Brak zebranych rekordów – sprawdź połączenie lub ogranicz CONCURRENCY.")

df_stats = pd.DataFrame(records).drop_duplicates()

# czyszczenie podstawowych typów
int_like = [
    "knockdowns","sig_strikes_succ","sig_strikes_att","total_strikes_succ","total_strikes_att",
    "takedown_succ","takedown_att","submission_att","reversals","ctrl_time_sec"
]
for c in int_like:
    if c in df_stats.columns:
        df_stats[c] = pd.to_numeric(df_stats[c], errors="coerce").fillna(0).astype(int)

# 4) zapis
df_stats.to_csv(OUTPUT_STATS_CSV, index=False)
df_stats.to_parquet(OUTPUT_STATS_PARQUET, index=False)

# 5) szybkie podsumowanie
n_fights = df_stats["fight_url"].nunique()
n_rows   = len(df_stats)
print(f"\n100%|{'█'*10}| {len(fight_urls)}/{len(fight_urls)} [{elapsed/60:0.2f} min]")
print("Summary:")
print(f" - UFC_fights_stats_data: ({n_rows:,}, {df_stats.shape[1]})")
print(f" - Unique fights covered: {n_fights:,} / expected {len(fight_urls):,}")
print(f"Saved → {OUTPUT_STATS_CSV}  |  {OUTPUT_STATS_PARQUET}")


Fight URLs total: 8324


Scraping fights (stats):   0%|          | 0/8324 [00:00<?, ?it/s]


100%|██████████| 8324/8324 [23.66 min]
Summary:
 - UFC_fights_stats_data: (16,606, 158)
 - Unique fights covered: 8,303 / expected 8,324
Saved → UFC_fights_stats_data.csv  |  UFC_fights_stats_data.parquet


## 7) UFC Fighters A–Z — URLs Snapshot  
**Purpose:** Enumerate all fighter profile URLs (alphabetical index) from UFCStats.  
**Inputs:** A–Z listing pages.  
**Outputs:** Dataframe of fighter profile URLs with slug/ID fields for joins.  
**Notes:** Skips dupes; resilient to occasional 404s or soft‑redirects.

In [2]:
# === UFC Fighters URLs Snapshot (A–Z) ===
# Wyjście: /kaggle/working/UFC_fighters_urls.(csv|parquet)

import os, sys, logging, time, re
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone
from pathlib import Path

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
from tqdm import tqdm

OUTDIR = Path("/kaggle/working")
OUTDIR.mkdir(exist_ok=True)
BASENAME = "UFC_fighters_urls"

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s", stream=sys.stdout)

LETTERS = list("abcdefghijklmnopqrstuvwxyz")  # możesz ograniczyć do testów, np. ["a","b"]

def http_session():
    s = requests.Session()
    retry = Retry(
        total=4, backoff_factor=0.8,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"])
    )
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

SESSION = http_session()
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; Kaggle-UFC-Scraper/1.0)"}

def fetch_letter(letter, retries=3, delay=5):
    url = f"http://ufcstats.com/statistics/fighters?char={letter}&page=all"
    for attempt in range(retries):
        try:
            r = SESSION.get(url, headers=HEADERS, timeout=15)
            if r.status_code == 429:
                time.sleep(delay)
                continue
            r.raise_for_status()
            return letter, r.text
        except Exception:
            time.sleep(delay)
    return letter, None

def extract_fighter_urls_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")
    # celujemy w linki "fighter-details" (najbardziej stabilny selektor)
    links = [a["href"].strip().rstrip("/") for a in soup.select('a.b-link[href*="fighter-details"]')]
    # czasem na stronie jest po kilka linków do tego samego fightera → dedup
    return list(dict.fromkeys(links))

def save_df(df: pd.DataFrame, name: str):
    csv_path = OUTDIR / f"{name}.csv"
    pq_path  = OUTDIR / f"{name}.parquet"
    df.to_csv(csv_path, index=False)
    try:
        df.to_parquet(pq_path, index=False)
    except Exception as e:
        logging.warning(f"Parquet save failed ({e}); trying fastparquet…")
        df.to_parquet(pq_path, index=False, engine="fastparquet")
    logging.info(f"Saved: {csv_path}  &  {pq_path}")

all_urls = []

with ThreadPoolExecutor(max_workers=6) as ex:
    futures = [ex.submit(fetch_letter, L) for L in LETTERS]
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Fetching A–Z pages"):
        letter, html = fut.result()
        if html:
            urls = extract_fighter_urls_from_html(html)
            all_urls.extend(urls)

all_urls = list(dict.fromkeys(all_urls))  # dedup z zachowaniem kolejności

df_furls = pd.DataFrame({"fighter_url": all_urls})
df_furls["fighter_url"] = df_furls["fighter_url"].astype(str).str.strip().str.rstrip("/")
df_furls["scraped_at"] = datetime.now(timezone.utc).isoformat()

save_df(df_furls, BASENAME)

print("Summary:")
print(" - UFC_fighters_urls:", df_furls.shape)


Fetching A–Z pages: 100%|██████████| 26/26 [00:05<00:00,  4.64it/s]

Summary:
 - UFC_fighters_urls: (4436, 2)


## 8) UFC Fighter Profiles — Adaptive & Resumable (async)  
**Purpose:** Scrape individual fighter profile pages for bio and career record, concurrently. Supports resume on failure.  
**Inputs:** Fighter profile URLs; concurrency/timeout settings.  
**Outputs:** Fighter‑level table (name, record, height/reach, stance, DOB, team, etc.) with normalized types.  
**Notes:** Adaptive retries and soft parsing for partially filled profiles; checkpointing/resume logic.

In [10]:
# === UFC Fighters PROFILE SCRAPER — adaptive + resumable ===
# In : /kaggle/working/UFC_fighters_urls.(csv|parquet)
# Out: /kaggle/working/UFC_fighters_data.csv | .parquet  (+ shardy w /kaggle/working/_fighters_shards)

import asyncio, aiohttp, random, re, time, math, os
from aiohttp import ClientTimeout
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm

# ---------- PATHS ----------
WORK = Path("/kaggle/working")
URLS_PQ  = WORK/"UFC_fighters_urls.parquet"
URLS_CSV = WORK/"UFC_fighters_urls.csv"
FINAL_CSV= WORK/"UFC_fighters_data.csv"
FINAL_PQ = WORK/"UFC_fighters_data.parquet"
SHARDS   = WORK/"_fighters_shards"; SHARDS.mkdir(exist_ok=True)

# ---------- PARAMS (startowe; skorygują się automatycznie) ----------
BATCH_SIZE   = 300
CONCURRENCY  = 5            # start
DELAY_RANGE  = (0.18, 0.42) # start
TIMEOUT      = 25
RETRIES      = 6
MIN_CONC     = 2
MAX_CONC     = 8

UA_POOL = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
]
LETTERS = list("abcdefghijklmnopqrstuvwxyz")
def headers():
    return {
        "User-Agent": random.choice(UA_POOL),
        "Accept": "text/html,application/xhtml+xml",
        "Accept-Language": random.choice(["en-US,en;q=0.9","en-GB,en;q=0.8","en;q=0.7"]),
        "Connection": "keep-alive",
        "Referer": f"http://ufcstats.com/statistics/fighters?char={random.choice(LETTERS)}&page=all",
    }

# ---------- PARSERY ----------
def parse_name(soup):
    t = soup.select_one("div.b-content__title h2") or soup.select_one("span.b-content__title-highlight")
    txt = t.get_text(" ", strip=True) if t else ""
    parts = [p for p in txt.split() if p]
    first = parts[0] if parts else None
    last  = " ".join(parts[1:]) if len(parts)>1 else "NULL"
    return first, last

def parse_record(soup):
    rec = soup.select_one("span.b-content__title-record")
    txt = rec.get_text(" ", strip=True) if rec else ""
    m = re.search(r"(\d+)-(\d+)-(\d+)\s*(?:\((\d+)\s*NC\))?", txt)
    if m:
        w,l,d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        nc = int(m.group(4)) if m.group(4) else 0
        return w,l,d,nc
    nums = [int(x) for x in re.findall(r"\d+", txt)]
    w = nums[0] if len(nums)>0 else 0
    l = nums[1] if len(nums)>1 else 0
    d = nums[2] if len(nums)>2 else 0
    nc= nums[3] if len(nums)>3 else 0
    return w,l,d,nc

def parse_dims_block(soup):
    height_cm = weight_lbs = reach_cm = None
    stance = "NULL"; dob = None
    for li in soup.select("li.b-list__box-list-item"):
        t = li.get_text(" ", strip=True)
        if t.startswith("Height:"):
            m = re.search(r"(\d+)\s*'\s*(\d+)\s*\"", t)
            if m: height_cm = int(round((int(m.group(1))*12 + int(m.group(2))) * 2.54))
        elif t.startswith("Weight:"):
            m = re.search(r"(\d+)", t)
            if m: weight_lbs = int(m.group(1))
        elif t.startswith("Reach:"):
            m = re.search(r"(\d+(?:\.\d+)?)", t)
            if m: reach_cm = int(round(float(m.group(1))*2.54))
        elif t.upper().startswith("STANCE:") or t.startswith("Stance:"):
            stance = t.split(":",1)[1].strip() or "NULL"
        elif t.startswith("DOB:"):
            raw = t.split(":",1)[1].strip()
            if raw and raw != "--":
                from datetime import datetime
                for fmt in ("%b %d, %Y", "%B %d, %Y"):
                    try:
                        dob = datetime.strptime(raw, fmt).strftime("%Y-%m-%d"); break
                    except: pass
    return height_cm, weight_lbs, reach_cm, stance, dob

def parse_career_stats(soup):
    out = { "fighter_SlpM":0.0, "fighter_Str_Acc":0.0, "fighter_SApM":0.0, "fighter_Str_Def":0.0,
            "fighter_TD_Avg":0.0, "fighter_TD_Acc":0.0, "fighter_TD_Def":0.0, "fighter_Sub_Avg":0.0 }
    for li in soup.select("ul.b-list__box-list.b-list__box-list_margin-top li.b-list__box-list-item"):
        txt = li.get_text(" ", strip=True)
        if ":" not in txt: continue
        k, v = [x.strip() for x in txt.split(":",1)]
        def pct(x):
            m = re.search(r"(\d+(?:\.\d+)?)\s*%", x); return float(m.group(1))/100.0 if m else 0.0
        def f(x): 
            try: return 0.0 if x in ["--",""] else float(x)
            except: return 0.0
        if   k == "SLpM":       out["fighter_SlpM"] = f(v)
        elif k == "Str. Acc.":  out["fighter_Str_Acc"] = pct(v)
        elif k == "SApM":       out["fighter_SApM"] = f(v)
        elif k in ("Str. Def","Str. Def."): out["fighter_Str_Def"] = pct(v)
        elif k == "TD Avg.":    out["fighter_TD_Avg"] = f(v)
        elif k == "TD Acc.":    out["fighter_TD_Acc"] = pct(v)
        elif k == "TD Def.":    out["fighter_TD_Def"] = pct(v)
        elif k == "Sub. Avg.":  out["fighter_Sub_Avg"] = f(v)
    return out

# ---------- IO ----------
def load_seed():
    if URLS_PQ.exists(): df = pd.read_parquet(URLS_PQ)
    elif URLS_CSV.exists(): df = pd.read_csv(URLS_CSV)
    else: raise FileNotFoundError("Brak UFC_fighters_urls.(csv|parquet)")
    col = "fighter_url" if "fighter_url" in df.columns else [c for c in df.columns if "url" in c.lower()][0]
    urls = (df[col].astype(str).str.strip().str.rstrip("/").dropna().unique().tolist())
    return urls

def load_scraped_set():
    # pozwala na wznowienie
    scraped = set()
    if FINAL_PQ.exists(): scraped |= set(pd.read_parquet(FINAL_PQ)["fighter_url"].astype(str))
    elif FINAL_CSV.exists(): scraped |= set(pd.read_csv(FINAL_CSV)["fighter_url"].astype(str))
    for p in SHARDS.glob("fighters_part_*.parquet"):
        try:
            scraped |= set(pd.read_parquet(p)["fighter_url"].astype(str))
        except: pass
    return set([u.strip().rstrip("/") for u in scraped])

def save_shard(df, idx):
    out = SHARDS/f"fighters_part_{idx:04d}.parquet"
    df.to_parquet(out, index=False)

def finalize_merge():
    parts = sorted(SHARDS.glob("fighters_part_*.parquet"))
    dfs = [pd.read_parquet(p) for p in parts] if parts else []
    if FINAL_PQ.exists():
        dfs.insert(0, pd.read_parquet(FINAL_PQ))
    elif FINAL_CSV.exists():
        dfs.insert(0, pd.read_csv(FINAL_CSV))
    if not dfs:
        raise RuntimeError("Brak danych do scalenia.")
    full = (pd.concat(dfs, ignore_index=True)
              .drop_duplicates(subset=["fighter_url"])
              .reset_index(drop=True))
    full.to_csv(FINAL_CSV, index=False)
    try:
        full.to_parquet(FINAL_PQ, index=False)
    except Exception:
        full.to_parquet(FINAL_PQ, index=False, engine="fastparquet")
    return full

# ---------- FETCH ----------
async def fetch_one(session, sem, url, delay_range, retries):
    url = url.strip().rstrip("/")
    async with sem:
        backoff = 1.1
        for attempt in range(retries):
            try:
                await asyncio.sleep(random.uniform(*delay_range))
                async with session.get(url, headers=headers()) as resp:
                    if resp.status in (429,403,500,502,503,504):
                        await asyncio.sleep(backoff*(attempt+1)); continue
                    resp.raise_for_status()
                    html = await resp.text()
                    if not html or len(html) < 1500:
                        await asyncio.sleep(backoff*(attempt+1)); continue
                    soup = BeautifulSoup(html, "html.parser")

                    f_name, l_name = parse_name(soup)
                    if not f_name:
                        await asyncio.sleep(backoff*(attempt+1)); continue

                    nn = soup.select_one("p.b-content__Nickname")
                    nickname = nn.get_text(strip=True) if nn and nn.get_text(strip=True) else "NULL"
                    h,w,r,stance,dob = parse_dims_block(soup)
                    W,L,D,NC = parse_record(soup)
                    stats = parse_career_stats(soup)

                    rec = {
                        "fighter_f_name": f_name,
                        "fighter_l_name": l_name if l_name else "NULL",
                        "fighter_nickname": nickname,
                        "fighter_height_cm": h,
                        "fighter_weight_lbs": w,
                        "fighter_reach_cm": r,
                        "fighter_stance": stance if stance else "NULL",
                        "fighter_dob": dob,
                        "fighter_w": W, "fighter_l": L, "fighter_d": D, "fighter_nc_dq": NC,
                        **stats,
                        "fighter_url": url
                    }
                    return rec
            except Exception:
                await asyncio.sleep(backoff*(attempt+1))
    return None

async def scrape_batch(urls, concurrency, delay_range):
    timeout = ClientTimeout(total=TIMEOUT)
    connector = aiohttp.TCPConnector(limit=concurrency*2, limit_per_host=concurrency, enable_cleanup_closed=True)
    sem = asyncio.Semaphore(concurrency)
    async with aiohttp.ClientSession(timeout=timeout, connector=connector) as session:
        tasks = [fetch_one(session, sem, u, delay_range, RETRIES) for u in urls]
        out = []
        for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc=f"Batch (C={concurrency}, delay={delay_range})"):
            rec = await fut
            if rec: out.append(rec)
        return out

def nb_run(coro):
    try:
        return asyncio.run(coro)
    except RuntimeError:
        import nest_asyncio; nest_asyncio.apply()
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(coro)

# ---------- MAIN ----------
all_urls = load_seed()
scraped = load_scraped_set()
todo = [u for u in all_urls if u not in scraped]
print(f"Total URLs : {len(all_urls)}")
print(f"Already have: {len(scraped)}")
print(f"To scrape  : {len(todo)}")

batch_idx = 0
concur, dly = CONCURRENCY, list(DELAY_RANGE)

while todo:
    sub = todo[:BATCH_SIZE]; todo = todo[BATCH_SIZE:]
    print(f"\n▶️ Batch {batch_idx+1}: {len(sub)} URLs | C={concur}, delay=({dly[0]:.2f},{dly[1]:.2f})")

    t0 = time.time()
    recs = nb_run(scrape_batch(sub, concur, tuple(dly)))
    dt = time.time()-t0

    df = pd.DataFrame(recs)
    succ = len(df); rate = succ / max(1,len(sub))
    print(f"   Collected {succ}/{len(sub)} ({rate*100:.1f}%) in {dt:.1f}s")
    if succ:
        # szybkie czyszczenie typów (bez forsowania Int64)
        for c in ["fighter_height_cm","fighter_weight_lbs","fighter_reach_cm","fighter_w","fighter_l","fighter_d","fighter_nc_dq"]:
            if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
        for c in ["fighter_SlpM","fighter_Str_Acc","fighter_SApM","fighter_Str_Def","fighter_TD_Avg","fighter_TD_Acc","fighter_TD_Def","fighter_Sub_Avg"]:
            if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
        save_shard(df, batch_idx)
    else:
        print("   (empty batch)")

    # adaptacja: jeśli <30% trafień → zwolnij; jeśli >80% → przyspiesz trochę
    if rate < 0.30:
        concur = max(MIN_CONC, concur-1)
        dly[0] = min(dly[0]*1.4, 0.8)
        dly[1] = min(dly[1]*1.4, 1.2)
        print(f"   ↓ Adapting: C={concur}, delay=({dly[0]:.2f},{dly[1]:.2f})")
        time.sleep(10)  # krótki chill po słabym batchu
    elif rate > 0.80 and concur < MAX_CONC:
        concur += 1
        dly[0] = max(0.10, dly[0]*0.9)
        dly[1] = max(0.18, dly[1]*0.9)
        print(f"   ↑ Adapting: C={concur}, delay=({dly[0]:.2f},{dly[1]:.2f})")

    batch_idx += 1

# ---------- MERGE ----------
final = finalize_merge()
print("\nSummary:")
print(" - UFC_fighters_data:", final.shape)
print(" - Saved →", FINAL_CSV.name, "|", FINAL_PQ.name)


Total URLs : 4436
Already have: 1201
To scrape  : 3235

▶️ Batch 1: 300 URLs | C=5, delay=(0.18,0.42)


Batch (C=5, delay=(0.18, 0.42)):   0%|          | 0/300 [00:00<?, ?it/s]

   Collected 300/300 (100.0%) in 34.8s
   ↑ Adapting: C=6, delay=(0.16,0.38)

▶️ Batch 2: 300 URLs | C=6, delay=(0.16,0.38)


Batch (C=6, delay=(0.162, 0.378)):   0%|          | 0/300 [00:00<?, ?it/s]

   Collected 300/300 (100.0%) in 26.9s
   ↑ Adapting: C=7, delay=(0.15,0.34)

▶️ Batch 3: 300 URLs | C=7, delay=(0.15,0.34)


...

   Collected 300/300 (100.0%) in 19.6s

▶️ Batch 10: 300 URLs | C=8, delay=(0.13,0.31)


Batch (C=8, delay=(0.13122, 0.30618)):   0%|          | 0/300 [00:00<?, ?it/s]

   Collected 300/300 (100.0%) in 21.0s

▶️ Batch 11: 235 URLs | C=8, delay=(0.13,0.31)


Batch (C=8, delay=(0.13122, 0.30618)):   0%|          | 0/235 [00:00<?, ?it/s]

   Collected 235/235 (100.0%) in 14.7s

Summary:
 - UFC_fighters_data: (4436, 21)
 - Saved → UFC_fighters_data.csv | UFC_fighters_data.parquet


## 9) Betting Odds — The Odds API → CSV & Parquet  
**Purpose:** Pull current and historical moneyline odds for UFC events using The Odds API.  
**Inputs:** The Odds API key (use environment variable in Kaggle `os.environ`), market/sport params.  
**Outputs:** Odds table with book, price, timestamp; saved to `/kaggle/working` in CSV and Parquet.  
**Notes:** Respect rate limits; never hard‑code private keys in public notebooks.

In [11]:
# === UFC Betting Odds (The Odds API) → CSV & Parquet ===
# In : Internet must be ON (Kaggle: enable “Internet”)
# Out: /kaggle/working/UFC_betting_odds_api.csv  |  .parquet
#
# About the API key (important):
# - Do NOT hardcode your private key in public code. Read it from an environment
#   variable instead (e.g., Kaggle → Add-ons → Secrets → create ODDS_API_KEY).
# - This is NOT an endorsement. The Odds API has historically offered a free tier
#   of ~500 requests/month, which is typically enough for personal snapshots.
#   Policies can change — check their site for current limits.
# - Writing individual scrapers for each bookmaker is brittle/time-consuming; this
#   API is used here purely as a pragmatic, single-source snapshot.

import os, time, json, datetime as dt
from pathlib import Path
from typing import Dict, Any, List

import requests
import pandas as pd

# --- konfig ---
OUTDIR   = Path("/kaggle/working"); OUTDIR.mkdir(exist_ok=True)
BASENAME = "UFC_betting_odds_api"
CSV_PATH = OUTDIR / f"{BASENAME}.csv"
PQ_PATH  = OUTDIR / f"{BASENAME}.parquet"

# API KEY: weź z env, a jak nie ma — użyj podanego klucza (uwaga: klucz w kodzie = ryzyko wycieku)
API_KEY = os.getenv("ODDS_API_KEY", "")

API_URL = "https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds"
PARAMS = {
    "regions": "us,uk,eu",
    "markets": "h2h",
    "oddsFormat": "decimal",
    "apiKey": API_KEY
}

BOOKMAKER_REGIONS = {
    "DraftKings": "us", "FanDuel": "us", "BetMGM": "us", "BetUS": "us", "BetOnline.ag": "us", "BetRivers": "us", "BetAnySports": "us",
    "888sport": "uk", "Betfair": "uk", "Betway": "uk", "Paddy Power": "uk", "Virgin Bet": "uk", "Grosvenor": "uk", "LiveScore Bet": "uk", "Matchbook": "uk",
    "Unibet": "eu", "Unibet (FR)": "eu", "Unibet (NL)": "eu", "Betclic (FR)": "eu", "LeoVegas": "eu", "Marathon Bet": "eu", "Nordic Bet": "eu", "Coolbet": "eu", "Betsson": "eu"
}

# --- pobranie z prostym retry ---
def fetch_odds(url: str, params: Dict[str, Any], retries: int = 3, backoff: float = 1.2):
    last_headers = {}
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=20)
            last_headers = r.headers
            if r.status_code == 200:
                return r.json(), last_headers
            # 429 / 5xx: spróbuj ponownie
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(backoff * (attempt + 1))
                continue
            # inne kody → przerwij
            raise RuntimeError(f"HTTP {r.status_code}: {r.text[:300]}")
        except requests.RequestException as e:
            if attempt == retries - 1:
                raise
            time.sleep(backoff * (attempt + 1))
    # jeśli tu dotarliśmy, coś nie tak
    raise RuntimeError(f"Fetch failed after {retries} attempts; last headers: {last_headers}")

data, hdrs = fetch_odds(API_URL, PARAMS)

# pokaż limity API (jeśli zwrócone)
rl_rem = hdrs.get("x-requests-remaining")
rl_used= hdrs.get("x-requests-used")
print(f"Rate limit → remaining: {rl_rem}, used: {rl_used}")

# --- flatten → lista rekordów ---
rows: List[Dict[str, Any]] = []
now_iso = dt.datetime.utcnow().isoformat()

for match in data:
    commence_iso = match.get("commence_time")
    # ISO → data (UTC)
    event_date = None
    if commence_iso:
        try:
            event_date = dt.datetime.fromisoformat(commence_iso.replace("Z", "+00:00")).date().isoformat()
        except Exception:
            event_date = commence_iso[:10]
    sport_key   = match.get("sport_key")
    sport_title = match.get("sport_title")
    event_id    = match.get("id")
    event_name  = match.get("home_team") or None  # dla MMA często brak "home/away", ale zostawiam dla zgodności
    teams       = match.get("teams") or []

    for bookmaker in match.get("bookmakers", []):
        title  = bookmaker.get("title")
        region = BOOKMAKER_REGIONS.get(title, "unknown")
        last_update = bookmaker.get("last_update")
        for market in bookmaker.get("markets", []):
            if market.get("key") != "h2h":
                continue
            outcomes = market.get("outcomes") or []
            if len(outcomes) != 2:
                continue

            f1 = outcomes[0].get("name")
            o1 = outcomes[0].get("price")
            f2 = outcomes[1].get("name")
            o2 = outcomes[1].get("price")

            rows.append({
                "sport_key": sport_key,
                "sport_title": sport_title,
                "event_id": event_id,
                "event_date": event_date,
                "commence_time_utc": commence_iso,
                "teams": ", ".join(teams) if teams else None,
                "fighter_1": f1,
                "fighter_2": f2,
                "odds_1": o1,
                "odds_2": o2,
                "bookmaker": title,
                "region": region,
                "bookmaker_last_update": last_update,
                "fetched_at_utc": now_iso,
            })

if not rows:
    print("Brak danych do zapisania (API zwróciło pustą listę).")

# --- DataFrame + typy ---
df = pd.DataFrame(rows)
if not df.empty:
    # porządkuj typy/liczby
    for c in ["odds_1","odds_2"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # zapis
    df.to_csv(CSV_PATH, index=False)
    try:
        df.to_parquet(PQ_PATH, index=False)
    except Exception:
        df.to_parquet(PQ_PATH, index=False, engine="fastparquet")

    # podsumowanie
    print("Summary:")
    print(" - rows              :", len(df))
    print(" - unique events     :", df["event_id"].nunique() if "event_id" in df else "n/a")
    print(" - unique bookmakers :", df["bookmaker"].nunique() if "bookmaker" in df else "n/a")
    print(f"Saved → {CSV_PATH.name}  |  {PQ_PATH.name}")
else:
    print("Pusta ramka — nic nie zapisano.")


Rate limit → remaining: 470, used: 30
Summary:
 - rows              : 657
 - unique events     : 73
 - unique bookmakers : 26
Saved → UFC_betting_odds_api.csv  |  UFC_betting_odds_api.parquet


## 10) Official UFC Rankings → CSV & Parquet  
**Purpose:** Scrape official rankings from `ufc.com/rankings` and produce a clean table suitable for joins.  
**Inputs:** Public rankings page.  
**Outputs:** Ranked fighters with division, rank, champion flag; saved as CSV and Parquet to `/kaggle/working`.  
**Notes:** Light HTML parsing with BeautifulSoup; defensive handling for divisions with variable champion/contender counts.

In [13]:
# === UFC Rankings → CSV & Parquet (no BigQuery) ===
# In : internet ON
# Out: /kaggle/working/UFC_rankings.csv | .parquet

import time, datetime as dt, re
from pathlib import Path
from typing import List, Dict, Any

import requests
from bs4 import BeautifulSoup
import pandas as pd

OUTDIR   = Path("/kaggle/working"); OUTDIR.mkdir(exist_ok=True)
BASENAME = "UFC_rankings"
CSV_PATH = OUTDIR / f"{BASENAME}.csv"
PQ_PATH  = OUTDIR / f"{BASENAME}.parquet"

URL = "https://www.ufc.com/rankings"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/124.0.0.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml",
}

def fetch_html(url: str, retries: int = 3, backoff: float = 1.3) -> str:
    last_err = None
    for attempt in range(retries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=20)
            if r.status_code == 200 and r.text and len(r.text) > 1500:
                return r.text
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(backoff * (attempt + 1))
                continue
            r.raise_for_status()
            return r.text
        except Exception as e:
            last_err = e
            time.sleep(backoff * (attempt + 1))
    raise RuntimeError(f"Fetch failed after {retries} attempts: {last_err}")

def clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_rankings(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, "html.parser")
    sections = soup.find_all("div", class_="view-grouping")
    out: List[Dict[str, Any]] = []
    ranking_date = dt.datetime.utcnow().date().isoformat()

    for sec in sections:
        # Nazwa kategorii (np. "Men's Pound-for-Pound Top Rank", "Flyweight", "Women's Strawweight", itd.)
        header = sec.find("div", class_="view-grouping-header")
        division = clean_text(header.get_text()) if header else "Unknown"

        # Mistrz (opcjonalnie, nie dotyczy P4P)
        champ = sec.select_one("div.rankings--athlete--champion")
        if champ:
            # imię/nazwisko bywają w h5 lub linku wewnątrz boxu
            name_tag = champ.find("h5") or champ.find("a")
            champion_name = clean_text(name_tag.get_text()) if name_tag else clean_text(champ.get_text())
            if champion_name:
                out.append({
                    "date": ranking_date,
                    "weightclass": division,
                    "fighter": champion_name,
                    "rank": 0
                })

        # Zawodnicy 1–15 (niektóre dywizje mogą mieć mniej)
        # Stabilny selektor: komórki tytułu wiersza tabeli
        fighter_cells = sec.select("td.views-field.views-field-title")
        if not fighter_cells:
            # fallback: linki z nazwiskami (na wypadek zmiany klas)
            fighter_cells = sec.select("table a[href*='/athlete/']")
        rank = 1
        for cell in fighter_cells:
            fighter_name = clean_text(cell.get_text())
            if not fighter_name:
                continue
            out.append({
                "date": ranking_date,
                "weightclass": division,
                "fighter": fighter_name,
                "rank": rank
            })
            rank += 1

    return out

# --- run ---
html = fetch_html(URL)
rows = parse_rankings(html)

df = pd.DataFrame(rows, columns=["date","weightclass","fighter","rank"]).drop_duplicates()
df = df.sort_values(["weightclass","rank","fighter"]).reset_index(drop=True)

# zapis
df.to_csv(CSV_PATH, index=False)
try:
    df.to_parquet(PQ_PATH, index=False)
except Exception:
    df.to_parquet(PQ_PATH, index=False, engine="fastparquet")

print("Summary:")
print(" - rows:", len(df))
print(" - divisions:", df["weightclass"].nunique() if not df.empty else 0)
print(" - sample:")
print(df.head(12))
print(f"Saved → {CSV_PATH.name}  |  {PQ_PATH.name}")


Summary:
 - rows: 208
 - divisions: 13
 - sample:
          date   weightclass              fighter  rank
0   2025-09-05  Bantamweight    Merab Dvalishvili     0
1   2025-09-05  Bantamweight        Sean O'Malley     1
2   2025-09-05  Bantamweight    Umar Nurmagomedov     2
3   2025-09-05  Bantamweight             Petr Yan     3
4   2025-09-05  Bantamweight       Cory Sandhagen     4
5   2025-09-05  Bantamweight          Song Yadong     5
6   2025-09-05  Bantamweight  Deiveson Figueiredo     6
7   2025-09-05  Bantamweight          Marlon Vera     7
8   2025-09-05  Bantamweight       Mario Bautista     8
9   2025-09-05  Bantamweight             Rob Font     9
10  2025-09-05  Bantamweight       Aiemann Zahabi    10
11  2025-09-05  Bantamweight         Henry Cejudo    11
Saved → UFC_rankings.csv  |  UFC_rankings.parquet
